In [ ]:
#@title Preparar notebook. { display-mode: "form" }

!pip install tmdbsimple

import concurrent.futures
import pandas as pd
from google.colab import drive
from datetime import datetime
import math
import threading
import time
import socket
import requests
import gzip
import json
import tmdbsimple as tmdb
import os
import numpy as np
import pickle
socket.setdefaulttimeout(15)

drive.mount('/content/drive')

#@markdown Inserte su api_key de TMDB y la ruta donde se guardará el dataset:
api_key =  '579dfb6c11b73263d6ce7e1f0844e60d' #@param {type: 'string'}
path = '/content/drive/Shareddrives/TFG_2021/Datos/Datasets/' #@param {type: 'string'}

In [ ]:
#@title Obtener ids actualizados de TMDB { display-mode: "form" }
url = 'https://files.tmdb.org/p/exports/movie_ids_' + datetime.now().strftime("%m_%d_%Y") + '.json.gz'
filename = url.split("/")[-1]
with open(filename, "wb") as f:
    r = requests.get(url)
    f.write(r.content)

with gzip.open(filename, 'rb') as f:
    file_content = f.read()

df_ids = pd.read_json(file_content, lines=True)

In [ ]:
#@title Multi-processing. { display-mode: "form" }

#@markdown Parámetros:
pos_ini =  0#@param {type:"integer"}
pos_fin =  10#@param {type:"integer"}
MAX_THREADS =  15#@param {type:"integer"}
all_df = False #@param {type:"boolean"}

if all_df:
  pos_ini = 0
  pos_fin = len(df_ids)
elif pos_ini < 0 or pos_ini > len(df_ids) or pos_ini > pos_fin or pos_fin > len(df_ids):
  print("Error en pos_ini y pos_fin.")

if MAX_THREADS < 1: 
  print("Error en MAX_THREADS.")

def download_url(url):
  resp_en = requests.get("https://api.themoviedb.org/3/movie/" + str(url) + "?api_key=" + api_key + "&append_to_response=credits&language=en-US")
  resp_es = requests.get("https://api.themoviedb.org/3/movie/" + str(url) + "?api_key=" + api_key + "&language=en-US&language=es-ES").json()

  respdict = resp_en.json()
  respdict['credits'] = [x['name'] for x in respdict['credits']['cast'] if x['known_for_department'] == 'Acting']
  respdict['genres'] = [x['name'] for x in respdict['genres']]
  respdict['title_es'] = resp_es['title']
  respdict['overview_es'] = resp_es['overview']
  
  resp_en._content = json.dumps(respdict).encode()
  fh.write(resp_en.content + b'\n')

  story_urls = list(df_ids['id']) if all_df else list(df_ids['id'][pos_ini:pos_fin])
threads = min(MAX_THREADS, len(story_urls))
if os.path.exists(path + 'df_peliculas.json'):
  os.remove(path + 'df_peliculas.json')

with open(path + 'df_peliculas.json', "wb") as fh:
  with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
    executor.map(download_url, story_urls)

df_peliculas = pd.read_json(path + 'df_peliculas.json', lines=True)

df_peliculas.rename(columns={'credits': 'actors'}, inplace=True)
df_peliculas.drop(['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'homepage', 'imdb_id', 'original_language', 'popularity', 'production_companies',
               'production_countries', 'revenue', 'spoken_languages', 'status', 'tagline', 'video', 'vote_average', 'vote_count', 'title'], axis=1, inplace=True)

df_peliculas.to_pickle(path + 'TMDB - ' + datetime.now().strftime("%d-%m-%Y - %H:%M:%S") + '.pkl', protocol=pickle.HIGHEST_PROTOCOL)